In [26]:
import socket
import numpy as np
import pandas as pd
import time
import pika
import json

credentials = pika.PlainCredentials('thomas', 'Test2011')
parameters = pika.ConnectionParameters(credentials=credentials, host='40.127.197.251')
connection = pika.BlockingConnection(parameters)

channel = connection.channel()
channel.queue_declare(queue='hello')

UDP_IP = "192.168.1.14"
UDP_PORT = 5555
CHUNKSIZE=1500
sock = socket.socket(socket.AF_INET, # Internet
                    socket.SOCK_DGRAM) # UDP

sock.bind((UDP_IP, UDP_PORT))

def divide_chunks(l, n):    
    # looping til length l
    for i in range(0, len(l), n):
        yield l[i:i + n] #How many elements each list should have

def chunk_to_df(buffer):
    received = "".join(buffer).split(",")[1:]
    chunks = list(divide_chunks(received, 4))
    measurements = np.vstack(chunks)
    df = pd.DataFrame(measurements, columns=["sensor", "x", "y", "z"])
    return df

def prep_df(df, msgpack=True):
    for c in df.columns:
        if c=="sensor":
            df[c] = pd.to_numeric(df[c], errors="coerce", downcast="unsigned")
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce", downcast="float")
    #df["sensor"] = df["sensor"].astype(np.int8)
    if df.shape[0]>CHUNKSIZE:
        df = df.iloc[:CHUNKSIZE]
    #else:
    #    df = df.append(pd.DataFrame(index=range(CHUNKSIZE-len(df)), columns=df.columns), ignore_index=True)
    #    df.fillna(method="pad", inplace=True)
    if msgpack:
        df = df.to_msgpack()
        return df
    else:
        return df

chunks = []
try:
    print("Receiving stream...")
    while True:
        data, addr = sock.recvfrom(512) 
        #print(data.decode("UTF-8"))
        df = chunk_to_df(data.decode("UTF-8"))
        chunk = prep_df(df, True)
        #print(chunk.shape)
        #print(chunk.head())
        channel.basic_publish(exchange='', routing_key='hello', body=chunk) 
        #print("Stream ended.")

except Exception:
    print("Closing connection.")
    sock.close()
    connection.close()

Receiving stream...


KeyboardInterrupt: 